Import Libraries

In [1]:
from pprint import pprint
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt
from scipy.io import loadmat
import sys
from tqdm import tqdm
from scipy.signal import find_peaks
import pyentrp.entropy as ent
from scipy.stats import pearsonr
from scipy import signal as sg
from scipy.signal import find_peaks
from scipy.signal import correlate
import random

Inititalize paths and if needed add custom SNOMED-CT codes as strings in the custom_disease variable, leave as [] if you want to choose 'disease_count' many top abnormalities from the dataset.

In [2]:
dataset_path = "/mnt/Velocity_Vault/ECG/Data/WFDB"
memmap_path='/mnt/Velocity_Vault/ECG/Dataset/'

custom_disease=[]
disease_count=7


# short_start=12700
# short_end=21838
# shorter_dataset_size=21838

Reads .hea and .mat files. A set of (hea file, mat file) is considered valid if both have the same name

In [3]:
import os

def find_hea_mat(folder_path):
    hea_list = []
    mat_list = []

    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is a .hea file
        if filename.endswith('.hea'):
            # Get the base name without the extension
            base_name = filename[:-4]
            mat_filename = base_name + '.mat'
            
            # Check if the corresponding .mat file exists
            mat_filepath = os.path.join(folder_path, mat_filename)
            if os.path.exists(mat_filepath):
                # Add the paths to the respective lists
                hea_list.append(os.path.join(folder_path, filename))
                mat_list.append(mat_filepath)

    return hea_list, mat_list

hea_array, mat_array=find_hea_mat(dataset_path)

print("Full Size : ",len(hea_array))


# hea_array=hea_array[short_start:short_end]
# mat_array=mat_array[short_start:short_end]

print("HEA Array:", len(hea_array))
print("MAT Array:",len(mat_array))


Full Size :  21838
HEA Array: 21838
MAT Array: 21838


Load the .hea files as a dictionary containing all the necessary information

In [4]:
import re

def parse_header_file(file_path):
    # Initialize a dictionary to store the parsed data
    data_dict = {}

    # Open and read the .hea file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse the first line for general information (ID, sample rate, etc.)
    first_line = lines[0].strip()
    first_line_parts = first_line.split()
    data_dict['Recording_ID'] = first_line_parts[0]
    data_dict['Num_Leads'] = int(first_line_parts[1])
    data_dict['Sampling_Rate_Hz'] = int(first_line_parts[2])
    data_dict['Num_Samples'] = int(first_line_parts[3])

    # Parse the subsequent lines for lead-specific information
    lead_info = []
    for i in range(1, data_dict['Num_Leads'] + 1):
        lead_line = lines[i].strip()
        lead_parts = lead_line.split()
        lead_data = {
            'File_Name': lead_parts[0],
            'Bit_Depth': lead_parts[1],
            'Gain_mV': lead_parts[2],
            'ADC_Resolution': int(lead_parts[3]),
            'Baseline': int(lead_parts[4]),
            'Signal_Offset': int(lead_parts[5]),
            'Lead_Type': lead_parts[6]
        }
        lead_info.append(lead_data)
    
    data_dict['Leads'] = lead_info

    # Parse the last few lines for patient information
    patient_info = {}
    for line in lines[data_dict['Num_Leads'] + 1:]:
        line = line.strip()
        if line.startswith('#Age:'):
            patient_info['Age'] = line.split(":")[1].strip()
        elif line.startswith('#Sex:'):
            patient_info['Sex'] = line.split(":")[1].strip()
        elif line.startswith('#Dx:'):
            patient_info['Diagnosis_Code'] = line.split(":")[1].strip()

    data_dict['Patient_Info'] = patient_info

    return data_dict

In [5]:
from tqdm import tqdm

hea_dict=[]

for i in tqdm(range(len(hea_array))):
    if hea_array[i]==None:
        continue
    hea_dict.append(parse_header_file(hea_array[i]))
    
print(len(hea_dict))
# pprint(hea_dict)

disease_dict=[]

for i in tqdm(range(len(hea_dict))):
    disease_dict.append((hea_dict[i]["Patient_Info"]["Diagnosis_Code"]).split(','))
    
print(len(disease_dict))
# pprint(disease_dict)


100%|██████████| 21838/21838 [00:02<00:00, 7810.35it/s]


21837


100%|██████████| 21837/21837 [00:00<00:00, 254224.58it/s]

21837


Shows the occurance of each abnormalities in descending order and chooses the abnormalities that are going to be classified

In [6]:
from collections import Counter

flattened_list = [number for sublist in disease_dict for number in sublist]
occurrences = Counter(flattened_list)

print("Number of Abnormalities found in Dataset : ",len(occurrences),"\n")

occurrences = occurrences.most_common()

# Display the occurrences
for number, count in occurrences:
    print(f"Dx: {number}, Occurrences: {count}")


chosen_disease=[number for number,_ in occurrences[:disease_count]]

if len(custom_disease)!=0:
    chosen_disease=custom_disease

print("Chosen Diseases : ",chosen_disease)




Number of Abnormalities found in Dataset :  50 

Dx: 426783006, Occurrences: 18092
Dx: 164865005, Occurrences: 5261
Dx: 39732003, Occurrences: 5146
Dx: 164951009, Occurrences: 3389
Dx: 164873001, Occurrences: 2359
Dx: 164934002, Occurrences: 2345
Dx: 164861001, Occurrences: 2175
Dx: 445118002, Occurrences: 1626
Dx: 164889003, Occurrences: 1514
Dx: 164884008, Occurrences: 1154
Dx: 713426002, Occurrences: 1118
Dx: 429622005, Occurrences: 1009
Dx: 427084000, Occurrences: 826
Dx: 270492004, Occurrences: 797
Dx: 698252002, Occurrences: 789
Dx: 427393009, Occurrences: 772
Dx: 55930002, Occurrences: 770
Dx: 426177001, Occurrences: 637
Dx: 164917005, Occurrences: 548
Dx: 713427006, Occurrences: 542
Dx: 164909002, Occurrences: 536
Dx: 67741000119109, Occurrences: 427
Dx: 284470004, Occurrences: 398
Dx: 428750005, Occurrences: 381
Dx: 54329005, Occurrences: 354
Dx: 47665007, Occurrences: 343
Dx: 164947007, Occurrences: 340
Dx: 10370003, Occurrences: 296
Dx: 59931005, Occurrences: 294
Dx: 4254190

The Pan Tompkins Algorithm approach of converting raw ECG signals into moving window signals so that r_peaks, p_peaks, t_peaks can be detected efficiently

In [7]:
class Pan_Tompkins_QRS:
    def __init__(self, fs):
        self.fs = fs  # Sampling frequency (fs) passed as an argument

    def band_pass_filter(self, signal):
        result = None
        sig = signal.copy()

        # Low-pass filter (11 Hz cutoff)
        for index in range(len(signal)):
            sig[index] = signal[index]
            if (index >= 1):
                sig[index] += 2*sig[index-1]
            if (index >= 2):
                sig[index] -= sig[index-2]
            if (index >= 6):
                sig[index] -= 2*signal[index-6]
            if (index >= 12):
                sig[index] += signal[index-12]

        result = sig.copy()

        # High-pass filter (5 Hz cutoff)
        for index in range(len(signal)):
            result[index] = -1*sig[index]
            if (index >= 1):
                result[index] -= result[index-1]
            if (index >= 16):
                result[index] += 32*sig[index-16]
            if (index >= 32):
                result[index] += sig[index-32]

        # Normalize the result
        max_val = max(max(result), -min(result))
        result = result / max_val
        return result

    def derivative(self, signal):
        result = signal.copy()
        for index in range(len(signal)):
            result[index] = 0
            if (index >= 1):
                result[index] -= 2*signal[index-1]
            if (index >= 2):
                result[index] -= signal[index-2]
            if (index >= 2 and index <= len(signal)-2):
                result[index] += 2*signal[index+1]
            if (index >= 2 and index <= len(signal)-3):
                result[index] += signal[index+2]
            result[index] = (result[index] * self.fs) / 8  # Use self.fs instead of annotation.fs
        return result

    def squaring(self, signal):
        result = signal.copy()
        for index in range(len(signal)):
            result[index] = signal[index]**2
        return result

    def moving_window_integration(self, signal):
        result = signal.copy()
        win_size = round(0.150 * self.fs)

        # If the signal length is smaller than the win_size, use the signal length
        if len(signal) < win_size:
            win_size = len(signal)

        sum = 0
        for j in range(win_size):
            sum += signal[j] / win_size
            result[j] = sum

        for index in range(win_size, len(signal)):
            sum += signal[index] / win_size
            sum -= signal[index - win_size] / win_size
            result[index] = sum

        return result

    def solve(self, signal):
        input_signal = signal.iloc[:, 1].to_numpy()

        # Bandpass Filter
        bpass = self.band_pass_filter(input_signal.copy())

        # Derivative Function
        der = self.derivative(bpass.copy())

        # Squaring Function
        sqr = self.squaring(der.copy())

        # Moving Window Integration Function
        mwin = self.moving_window_integration(sqr.copy())

        return mwin


# Update process_ecg to adjust signal using Signal_Offset and Gain_mV from hea_dict
def preprocess_ecg(hea_dict, raw_ecg):
    #return raw_ecg
    # Create a dictionary of offsets and gains for each lead
    leads_info = hea_dict['Leads']
    
    processed_signals = []
    
    for i, lead in enumerate(leads_info):
        signal_offset = lead['Signal_Offset']
        gain_mv = int(lead['Gain_mV'].replace("/mv", ""))  # Gain in mV
        
        # Adjust the raw ECG signal for this lead by applying offset and gain
        adjusted_signal = (raw_ecg[i] + signal_offset) * gain_mv / 1000  # Scale to mV
        
        processed_signals.append(adjusted_signal)
    
    return processed_signals

def process_ecg(hea_dict, signal):
    # Load the .mat ECG signal data
    # Preprocess the ECG signals using the header information
    processed_signals = preprocess_ecg(hea_dict, signal)

    # Extract fs (sampling frequency) from the header
    fs = hea_dict['Sampling_Rate_Hz']  # Assuming it's present in the header file
    #print("Fs : ",fs)
    # Initialize QRS detector with fs
    QRS_detector = Pan_Tompkins_QRS(fs)

    # Solve using Pan-Tompkins algorithm for all signals
    processed_signals_qrs = [QRS_detector.solve(pd.DataFrame({'Time': range(len(signal)), 'ECG': signal})) for signal in processed_signals]

    return processed_signals_qrs

Functions to calculate the wide features of an ECG signal - Age, Gender (Male:0, Female:1), Max HR, Mean HR, Min HR, P wave correlation coefficient, RMSSD, RR interval (Mean), RR interval (Median), RR interval Fisher information, T wave permutation entropy (Median), T wave permutation entropy (STD)

In [8]:



def load_mat_file(mat_path):
    # Load the .mat file using scipy.io
    mat_data = scipy.io.loadmat(mat_path)
    # Assuming the ECG signal is stored in 'ecg' key or similar
    ecg_signal = mat_data['val']  # Replace 'ecg' with the actual key if different
    return ecg_signal

def calculate_rr_intervals(r_peaks, samp_freq):
    """
    Calculate RR intervals (differences between successive R-peaks).
    
    :param r_peaks: List or array of R-peak positions (indices).
    :param samp_freq: Sampling frequency of the ECG signal (in Hz).
    :return: List of RR intervals in milliseconds.
    """
    # Calculate the differences between successive R-peaks (in samples)
    rr_intervals_samples = np.diff(r_peaks)
    
    # Convert RR intervals from samples to milliseconds
    rr_intervals_ms = rr_intervals_samples / samp_freq * 1000  # Multiply by 1000 to convert to ms
    
    return rr_intervals_ms

def compute_mpe(signal, m, delay, scales):
    """
    Compute Multiscale Permutation Entropy (MPE) for a given signal.
    
    :param signal: 1D array or list of signal values.
    :param m: Embedding dimension for PE.
    :param delay: Time delay for PE.
    :param scales: List of scales for multiscale PE calculation.
    :return: List of MPE values for each scale.
    """
    mpe_values = []
    for scale in scales:
        # Coarse-grain the signal for the current scale
        coarse_grained = np.mean(signal[:len(signal)//scale * scale].reshape(-1, scale), axis=1)
        # Compute Permutation Entropy for the coarse-grained signal
        mpe_values.append(ent.permutation_entropy(coarse_grained, order=m, delay=delay))
    return mpe_values

def t_wave_mpe(ecg_signal, t_peaks, fs=500, window_size=0.2, m=3, delay=1, scales=[1, 2, 3, 4, 5]):
    """
    Compute T-wave Multiscale Permutation Entropy (MPE) for an ECG signal.
    
    :param ecg_signal: The ECG signal (array or list).
    :param t_peaks: List of indices of T-peaks in the ECG signal.
    :param fs: Sampling frequency of the signal (in Hz).
    :param window_size: Time window around T-peak to extract T-wave (in seconds).
    :param m: Embedding dimension for PE.
    :param delay: Time delay for PE.
    :param scales: List of scales for multiscale PE calculation.
    :return: List of MPE values for each T-wave segment.
    """
    t_wave_mpe_values = []
    half_window_samples = int(window_size * fs / 2)  # Half-window size in samples
    
    for t_peak in t_peaks:
        # Define the window around the T-peak
        start_idx = max(0, t_peak - half_window_samples)
        end_idx = min(len(ecg_signal), t_peak + half_window_samples)
        
        # Extract the T-wave segment
        t_wave_segment = ecg_signal[start_idx:end_idx]
        
        # Resample the segment to a fixed length (if needed)
        if len(t_wave_segment) != 2 * half_window_samples:
            t_wave_segment = np.pad(
                t_wave_segment, 
                (0, 2 * half_window_samples - len(t_wave_segment)), 
                'constant'
            )
        
        # Calculate Multiscale Permutation Entropy for the T-wave segment
        mpe_values = compute_mpe(t_wave_segment, m=m, delay=delay, scales=scales)
        t_wave_mpe_values.append(mpe_values)
    
    return np.std(t_wave_mpe_values),np.mean(t_wave_mpe_values)

def calculate_rr(rr_intervals):
    """
    Calculate the RMSSD (Root Mean Square of Successive Differences) from RR intervals.

    :param rr_intervals: List or array of RR intervals in milliseconds.
    :return: RMSSD value.
    """
    
    if len(rr_intervals)==0:
        return 0.0,0.0,0.0
    
    # Calculate successive differences of RR intervals
    diff_rr = np.diff(rr_intervals)
    
    # Square the differences
    squared_diff_rr = diff_rr ** 2
    
    # Calculate the mean of the squared differences
    mean_squared_diff = np.mean(squared_diff_rr)
    
    # Return the square root of the mean
    rmssd = np.sqrt(mean_squared_diff)
    rrmed=np.median(diff_rr)
    rrmin=np.min(diff_rr)
    return rmssd,rrmed,rrmin

def find_p_peaks(ecg_signal, r_peaks, sampling_rate=500, p_wave_window=150):
    """
    Detect P-peaks in an ECG signal based on R-peak locations.

    :param ecg_signal: 1D array or list of the ECG signal.
    :param r_peaks: List of R-peak indices.
    :param sampling_rate: Sampling rate of the ECG signal in Hz.
    :param p_wave_window: Time window before each R-peak to search for P-peaks (in ms).
    :return: List of P-peak indices.
    """
    # Convert the P-wave window from ms to samples
    p_wave_window_samples = int((p_wave_window / 1000) * sampling_rate)
    p_peaks = []

    for r_peak in r_peaks:
        # Define the P-wave search region (before the R-peak)
        start_idx = max(0, r_peak - p_wave_window_samples)
        end_idx = r_peak

        # Extract the segment of the signal for P-wave detection
        p_wave_region = ecg_signal[start_idx:end_idx]

        # Detect peaks in the P-wave region
        peaks, _ = find_peaks(p_wave_region)

        if len(peaks) > 0:
            # Find the peak with the maximum amplitude (assume it corresponds to the P-wave)
            p_peak_idx = peaks[np.argmax(p_wave_region[peaks])]
            p_peaks.append(start_idx + p_peak_idx)

    return p_peaks

def find_t_peaks(ecg_signal, r_peaks, fs=500, t_window=(0.15, 0.4)):
    """
    Find T-peaks in an ECG signal.

    :param ecg_signal: The ECG signal (array or list).
    :param r_peaks: Indices of detected R-peaks.
    :param fs: Sampling frequency of the signal (in Hz).
    :param t_window: Time window after R-peaks to search for T-peaks (in seconds).
    :return: List of indices of T-peaks.
    """
    t_peaks = []
    search_start_offset = int(t_window[0] * fs)  # Convert seconds to samples
    search_end_offset = int(t_window[1] * fs)    # Convert seconds to samples

    for r_peak in r_peaks:
        # Define the search window for the T-wave
        search_start = r_peak + search_start_offset
        search_end = r_peak + search_end_offset

        # Ensure the search window is within signal bounds
        if search_start >= len(ecg_signal) or search_end > len(ecg_signal):
            break

        # Extract the segment where the T-wave is expected
        t_wave_segment = ecg_signal[search_start:search_end]

        # Find the index of the maximum amplitude in the segment
        t_peak_relative = np.argmax(t_wave_segment)
        t_peak = search_start + t_peak_relative

        t_peaks.append(t_peak)

    return t_peaks

def compute_p_wave_correlation(ecg_signal, p_peaks, window_size=50):
    """
    Compute the P-wave correlation coefficient between individual P-waves and the template P-wave.

    :param ecg_signal: The ECG signal (array or list).
    :param p_peaks: List of P-peak indices.
    :param window_size: The window size around each P-peak to define the P-wave (in samples).
    :return: List of correlation coefficients for each P-wave with the template P-wave.
    """
    # Step 1: Extract P-wave segments
    p_waves = []
    for p_peak in p_peaks:
        start_idx = max(0, p_peak - window_size // 2)
        end_idx = min(len(ecg_signal), p_peak + window_size // 2)
        p_wave = ecg_signal[start_idx:end_idx]
        
        # Zero-pad if the segment size is smaller than the window size
        if len(p_wave) < window_size:
            p_wave = np.pad(p_wave, (0, window_size - len(p_wave)), mode='constant')
        
        p_waves.append(p_wave)
    
    # Step 2: Compute the template P-wave (average of all P-waves)
    p_wave_template = np.mean(p_waves, axis=0)
    
    # Step 3: Compute correlation coefficients
    correlation_coefficients = []
    for p_wave in p_waves:
        if np.all(p_wave == p_wave[0]):
            corr=0
        else:
            corr, _ = pearsonr(p_wave, p_wave_template)
            
        correlation_coefficients.append(corr)
    
    return np.mean(correlation_coefficients)

def calculate_rr_fisher_information(rr_intervals):
    """
    Calculate the Fisher Information for the mean of the RR intervals.
    
    The Fisher Information for the mean of a normal distribution is given by:
        Fisher Information = 1 / variance
    
    :param rr_intervals: List or array of RR intervals in milliseconds.
    :return: Fisher Information value.
    """
    # Calculate the variance of the RR intervals
    variance = np.var(rr_intervals)
    
    # Fisher Information is the inverse of the variance
    fisher_information = 1 / variance if variance > 0 else 0
    
    return fisher_information

def check_gender(s):
    s = s.lower()  # Convert the string to lowercase for case-insensitive comparison
    if 'male' in s:
        return 0
    elif 'female' in s:
        return 1
    else:
        return random.choice([0, 1])

def check_age(s):
    # Try to find an integer in the string
    numbers = [int(word) for word in s.split() if word.isdigit()]
    
    if numbers:
        return numbers[0]  # Return the first integer found
    else:
        return random.randint(20, 95)
    
def get_10_seconds(signal,fs=500):
    sampled=[]
    for lead in signal:
        sampled.append(get_random_10s_segment(lead,sampling_rate=fs))
    return np.array(sampled)
        
        
def get_random_10s_segment(ecg_signal, sampling_rate=500):
    # Calculate the number of samples for 15 seconds
    num_samples = int(10 * sampling_rate)
    
    # Check if the signal is shorter than 15 seconds
    if len(ecg_signal) < num_samples:
        # Apply zero-padding
        padded_signal = np.zeros(num_samples)
        padded_signal[:len(ecg_signal)] = ecg_signal
        return padded_signal
    elif len(ecg_signal)==num_samples:
        return ecg_signal
    else:
        return ecg_signal[0:num_samples]


Function that loads the signals and returns the raw signals, wide features and labels.

In [9]:
def calculate_labels(file_index,hea_array,mat_array,valid_disease):
    hea_dict=parse_header_file(hea_array[file_index])
    signal_org = load_mat_file(mat_array[file_index])
    
    #print(signal_org.shape)
    
    disease=(hea_dict["Patient_Info"]["Diagnosis_Code"]).split(',')
    age=hea_dict["Patient_Info"]["Age"]
    gender=hea_dict["Patient_Info"]["Sex"]
    fs = hea_dict['Sampling_Rate_Hz']
    
    presence=[]
    
    for dis in valid_disease:
        if any(dis.lower() in s.lower() for s in disease):
            presence.append(1)
        else:
            presence.append(0)
            
    # if presence==[0 for _ in range(len*valid_disease)]:
    #     return [],[],[]
    
    signal_sampled= get_10_seconds(signal_org,fs=fs)
    
    signal=process_ecg(hea_dict,signal_sampled)
    
    features=[]
    for i in range(len(signal)):
        feat=calculate_each_signal(signal[i],fs)
        features.append(feat)
        
    features = list(map(list, zip(*features)))
    
    classify=[np.mean(fe) for fe in features]
    ag=check_age(age)
    ge=check_gender(gender)
    classify.append(ag)
    classify.append(ge)
    

    
    return signal_sampled,classify,presence
    
def calculate_each_signal(signal,fs):
    samp_freq=fs
    
    r_peaks,_ = find_peaks(signal, distance=200, height=0.5)
    r_peaks = np.array(r_peaks)
    r_peaks = r_peaks[r_peaks > 0]
    
    if len(r_peaks)==0:
        return [0.0 for _ in range(10)]
    
    p_peaks = find_p_peaks(signal, r_peaks, sampling_rate=fs)

    t_peaks=find_t_peaks(signal,r_peaks, fs=fs)
    
    r_peaks=r_peaks[1:]
    p_peaks=p_peaks[1:]
    t_peaks=t_peaks[1:]
    
    # Calculate the heart rate
    h_rates=np.diff(r_peaks)
    
    
    if len(h_rates)==0:
        h_mean=0
        h_max=0
        h_min=0
    else:
        h_rates=h_rates*2
        h_mean = (60*fs)/np.average(h_rates)
        h_max= (60*fs)/np.min([x for x in h_rates if x != 0])
        h_min = (60*fs)/np.max(h_rates)
    
    rr_interval=calculate_rr_intervals(r_peaks,samp_freq)
    std_mpe,median_mpe=t_wave_mpe(signal, t_peaks)
    rmssd,rrmed,rrmin = calculate_rr(r_peaks)
    
    
    p_wave_corrs = compute_p_wave_correlation(signal, p_peaks)
    
    fisher=calculate_rr_fisher_information(rr_interval)
    
    features=[h_min,std_mpe,h_max,median_mpe,rmssd,p_wave_corrs,rrmed,h_mean,fisher,rrmin]
    features=[0 if (np.isnan(x) or np.isinf(x)) else x for x in features]

    return features

Some times files may not contain any information or maybe corrupted, those are filtered out here and only valid files are going to be computed

In [10]:

dataset_size=min(len(hea_array),len(mat_array))

size=0
index_dict={}
for test_number in range(dataset_size):
    if hea_array[test_number]==None:
        continue
    
    # hea_dict=parse_header_file(hea_array[test_number])
    # disease=hea_dict["Patient_Info"]["Diagnosis_Code"].split(',')
    
    # presence=False
    
    # for dis in chosen_disease:
    #     if any(dis.lower() in s.lower() for s in disease):
    #         presence=True
            
    # if not presence:
    #     continue
            
    
    index_dict[test_number]=size
    size+=1
    
print('Dataset Size : ',size)

Dataset Size :  21837


Creating the dataset within the applicable size

In [11]:

ecg_signal=np.memmap(memmap_path+'ecg_signal', dtype='int16', mode='w+', shape=(size, 12,5000))
features=np.memmap(memmap_path+'features', dtype='float32', mode='w+', shape=(size, 12))
labels=np.memmap(memmap_path+'labels', dtype='int', mode='w+', shape=(size, len(chosen_disease)))

# features=np.zeros((size, 12), dtype=np.float32)
# labels=np.zeros((size, len(chosen_disease)), dtype=int)

for test_number in tqdm(range(dataset_size)):
    if hea_array[test_number]==None:
        continue
    sig,feat,lab=calculate_labels(test_number,hea_array,mat_array,chosen_disease)
    
    ecg_signal[index_dict[test_number]]=sig    
    features[index_dict[test_number]]=feat
    labels[index_dict[test_number]]=lab

100%|██████████| 21838/21838 [1:16:47<00:00,  4.74it/s]


Priniting average features over the entire dataset

In [12]:
average_feature = np.mean(features, axis=0)
average_label = np.mean(labels, axis=0)

feat_dict_key = ['Min HR', 'T wave permutation entropy (STD)',
            'Max HR', 'T wave permutation entropy (Median)', 'RMSSD',
            'P wave correlation coefficient', 
            'RR interval (Median)', 'Mean HR', 'RR interval Fisher information',
            'RR interval (Mean)','Age','Gender (Male:0, Female:1)']

# Create a dictionary to map feature names to their corresponding values
feature_dict = dict(zip(feat_dict_key, average_feature))
label_dict = dict(zip(chosen_disease, average_label))

print("Average Features \n")
pprint(feature_dict)
print("\nAverage Labels \n")
pprint(label_dict)

Average Features 

{'Age': 59.827267,
 'Gender (Male:0, Female:1)': 0.0,
 'Max HR': 74.05248,
 'Mean HR': 55.87401,
 'Min HR': 39.076946,
 'P wave correlation coefficient': 0.60543877,
 'RMSSD': 275.7696,
 'RR interval (Mean)': 202.63823,
 'RR interval (Median)': 255.66045,
 'RR interval Fisher information': 8.3664374e-05,
 'T wave permutation entropy (Median)': 1.7843825,
 'T wave permutation entropy (STD)': 0.29116395}

Average Labels 

{'164861001': 0.099601593625498,
 '164865005': 0.24092137198333105,
 '164873001': 0.1080276594770344,
 '164934002': 0.10738654577093923,
 '164951009': 0.15519531071117829,
 '39732003': 0.2356550808261208,
 '426783006': 0.8285020836195448}


Printing the shapes for future uses

In [13]:
print(ecg_signal.shape)
print(features.shape)
print(labels.shape)

(21837, 12, 5000)
(21837, 12)
(21837, 7)
